<a href="https://colab.research.google.com/github/anqi-guo/duplicated_complaints_identification/blob/main/5_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
orig_df = pd.read_csv('/content/drive/My Drive/work/重复线索识别/raw_data/raw_data.csv')

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install hanlp[full]
!pip install sentence_transformers

In [ ]:
import hanlp
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
ner = hanlp.load(hanlp.pretrained.ner.MSRA_NER_ELECTRA_SMALL_ZH)

import regex as re
import jieba

def extract_problem(text):
  # 删掉标点符号
  text = re.sub(r'\p{P}+', '', text) 
  # 删掉空格、换行符、信息保密、信息不保密、距离、时间、数字、投诉人信息
  text = re.sub(r'(\s|\n|信息[不]?保密|\d+[m米年月日]?|(.{1}先生|.{1}女士|市民)(来电)?反映)', '', text) 

  if '问题描述' in text and '诉求目的' in text:
    text = re.search(r'(?<=问题描述).*(?=诉求目的)', text).group()

  # 删掉停用词
  with open('/content/drive/My Drive/work/重复线索识别/stopwords/cn_stopwords.txt') as f:
    stopwords = f.read().splitlines() 

  seg_list = jieba.cut(text)

  result = ''.join([s for s in seg_list if s not in stopwords])
  return result

100%  43.5 MiB   3.5 MiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/tok/coarse_electra_small_20220220_013548.zip to /root/.hanlp/tok
100%  26.7 KiB  26.7 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/utils/char_table_20210602_202632.json.zip to /root/.hanlp/utils
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/transformers/electra_zh_small_20210706_125427.zip to /root/.hanlp/transformers
100%  45.0 MiB  13.4 MiB/s ETA:  0 s [=========================================]
Decompressing /root/.hanlp/ner/msra_ner_electra_small_20220215_205503.zip to /root/.hanlp/ner


In [ ]:
df = orig_df[~orig_df['XSNR'].str.contains('前期工单|原工单|已有工单|前单号|类似工单')]
df.shape

(71849, 17)

In [ ]:
df['problem'] = df['XSNR'].progress_apply(extract_problem)

  0%|          | 0/71849 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# load fine-tuned model
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('/content/drive/My Drive/work/重复线索识别/models/sb')

for i in df['XZQH'].unique():
    print(i)
    df_i = df[df['XZQH']==i].reset_index(drop=True)
    sentences = df_i['problem'].tolist()
    paraphrases = util.paraphrase_mining(model, sentences, show_progress_bar=True, max_pairs=5000, top_k=10)

    similar_sentence_pairs = []
    for paraphrase in paraphrases:
        score, i, j = paraphrase
        if i != j and score > .5:
            df_s = pd.DataFrame(data={'score':[score], 'sentence1':[df_i.loc[i,'XSNR']], 'sentence2':[df_i.loc[j, 'XSNR']], 'bh1': [df_i.loc[i, 'BH']], 'bh2': [df_i.loc[j, 'BH']]})
            df_s.to_csv(f'/content/drive/My Drive/work/重复线索识别/outputs/similar_sentence_pairs.csv', mode='a', header=False, index=False)

淮阴区


Batches:   0%|          | 0/214 [00:00<?, ?it/s]

新港街道


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

市工业园区


...

石湖镇


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

nan


Batches: 0it [00:00, ?it/s]

RuntimeError: ignored

In [ ]:
# 如果文本中包含工单号码则提取工单号码
from tqdm.auto import tqdm
tqdm.pandas()

def get_gongdan_number(text):
  # 删掉标点符号
  text = re.sub(r'\p{P}+', '', text)
  # 提取工单号
  gongdan_number = re.findall('(?<=(?:前期工单|原工单|已有工单|前单号))[a-zA-Z0-9]+', text)
  gongdan_number2 = re.findall('[a-zA-Z0-9]{7,50}', text)

  if gongdan_number:
    return gongdan_number[0]
  elif ('前期工单' in text or '原工单' in text or '已有工单' in text or '前单号' in text) and gongdan_number2:
    return gongdan_number2[0]
  else:
    return None

orig_df['前工单号'] =  orig_df['XSNR'].progress_apply(get_gongdan_number)

  0%|          | 0/80101 [00:00<?, ?it/s]

In [ ]:
orig_df[orig_df['XSNR'].str.contains('前期工单|原工单|已有工单|前单号')][['BH','XSBH','XSNR','前工单号']].sample(10)

,BH,XSBH,XSNR,前工单号
16898,1583535733340064212992,202003040022,前期工单：DH9932080020022900083 ，熊先生来电反映：徐溜镇镇上逢集，农贸...,DH9932080020022900083
44721,1624666369181046039040,202106170095,1、问题描述：前期工单DH9932080021060502947，大商城小区南门贾味烧烤，2...,DH9932080021060502947
17957,1585177863938047243264,202003170078,前期工单：DH9932080019021100216，桂先生来电反映：富豪花园A区门面房“玉...,DH9932080019021100216
30904,1600988947499048492544,202009160059,前期工单：DH9932080020091000069，淮阴区樱花园2期西门口南边的唯一一个篮...,DH9932080020091000069
52576,1632524356771044273664,202109170090,1、问题描述：前期工单：DH9932080021072403052，萧湖公园状元阁前侧萧湖度...,DH9932080021072403052
2718,1553487692765093564928,201903190018,一、市民不认可，表示该水泥厂每天7点至18：30分期间一直正常生产。（市平台退单原因）\n二...,None
65492,1645571173644133468160,202202080048,1、问题描述：前期工单：DH9932080022012505109，市民来电反映：涟水县高沟...,DH9932080022012505109
48115,1628637111476047472640,202108070020,1、问题描述：前期工单：DH9932080021080503242，淮阴区汽车北站的“名珠家...,DH9932080021080503242
19018,1586473670029046899200,202003270067,前期工单：DH9932080020032201562 ，市民来电反映：荷湖星城西北门南侧“中...,DH9932080020032201562
27309,1596755159998042086400,202008030032,市民来电反映：前期工单：DH9932080020072100357 黎城镇润康天成小区离旺达...,DH9932080020072100357
